In [1]:
import pandas as pd
import numpy as np
import recommendations
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
df = pd.read_csv('../datasets/SEHIR/processed_dataset.csv')
df = df[['Course Code', 'Student Number', 'Letter Grade', 'Semester', 'Course Credit']]

In [3]:
grade_mapper = {'A+': 4.1, 'A': 4.0, 'A-': 3.7, 'B+': 3.3, 'B': 3.0, 'B-': 2.7, 'C+': 2.3, 
                'C': 2.0, 'C-': 1.7, 'D+': 1.3, 'D': 1.0, 'D-': 0.5, 'F': 0.0}

In [4]:
semester_data = {}
course_data = {}
for row_idx in df.index:
    course_code = df.iloc[row_idx, 0]
    student_number = df.iloc[row_idx, 1]
    letter_grade = df.iloc[row_idx, 2]
    semester = df.iloc[row_idx, 3]
    course_credit = df.iloc[row_idx, 4]
    
    semester_data.setdefault(semester, {})
    semester_data[semester].setdefault(student_number, {})
    semester_data[semester][student_number][course_code] = grade_mapper[letter_grade]
    course_data[course_code] = course_credit

In [5]:
def get_gpa(semester_data, student):
    total_weight = 0
    total_credit = 0
    
    for course in semester_data[student]:
        total_weight += semester_data[student][course] * course_data[course]
        total_credit += course_data[course]
    
    return total_weight / total_credit

In [6]:
def get_grade_stats(semester_data, student):
    grade_list = []
    
    for course in semester_data[student]:
        numerical_grade = semester_data[student][course]
        grade_list.append(numerical_grade)
    
    mean = np.mean(grade_list)
    std_dev = np.std(grade_list)
    
    return mean, std_dev

In [7]:
def get_errors(train_semester, test_semester, sim):
    y_true = []
    y_pred = []
    gpa = {}
    
    for student in train_semester:
        gpa[student] = get_gpa(train_semester, student)
        
    for student in train_semester:
        recommended_courses = {}
        recs = recommendations.getRecommendations(train_semester, student, sim, dgpa=True, gpa=gpa, delta=0.7)
            
        for rec_grade, rec_course in recs:
            recommended_courses.setdefault(rec_course, rec_grade)
            
        gpa.setdefault(student, get_gpa(train_semester, student))
        
        if student not in test_semester:
            continue
        
        mean, std_dev = get_grade_stats(train_semester, student)
            
        for course_code in test_semester[student]:
            if course_code in recommended_courses:
                rec_grade = recommended_courses[course_code]
                if rec_grade < mean - (2 * std_dev) or rec_grade > mean + (2 * std_dev):
                    continue
                y_pred.append(rec_grade)
            else:
                rec_grade = gpa[student]
                if rec_grade < mean - (2 * std_dev) or rec_grade > mean + (2 * std_dev):
                    continue
                y_pred.append(rec_grade)
            y_true.append(test_semester[student][course_code])
        
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    
    return rmse, mae

In [8]:
def predict(semester_data, sim):
    errors = {'rmse': [], 'mae': []}
    train_semester = {}
    for i in range(1, len(sorted(semester_data))):
        new_semester = semester_data[sorted(semester_data)[i-1]]
        for student in new_semester:
            if student in train_semester:
                train_semester[student].update(new_semester[student])
            else:
                train_semester[student] = new_semester[student]
        test_semester = semester_data[sorted(semester_data)[i]]

        # making predictions for every student in training semester
        rmse, mae = get_errors(train_semester, test_semester, sim)

        errors['rmse'].append(rmse)
        errors['mae'].append(mae)
    
    return errors

In [9]:
def plot_errors(errors, sim):
    x = np.arange(7)
    fig, ax = plt.subplots(figsize=(11, 5.5))
    rects1 = ax.bar(x - 0.35/2, errors['rmse'], 0.35, label='RMSE')
    rects2 = ax.bar(x + 0.35/2, errors['mae'], 0.35, label='MAE')
    for i, v in enumerate(errors['rmse']):
        ax.text(i-0.36, v + 0.01, str(round(v, 2)), fontfamily='serif', fontsize=13, fontweight='bold')
    for i, v in enumerate(errors['mae']):
        ax.text(i-0.01, v + 0.01, str(round(v, 2)), fontfamily='serif', fontsize=13, fontweight='bold')
    ax.set_ylabel('Error', fontsize=22, fontfamily='serif')
    ax.set_title('Error Metrics with N training semesters ({})'.format(sim), fontsize=22, fontfamily='serif')
    ax.set_xticks(x)
    ax.set_xticklabels(['N={}'.format(i+1) for i in range(7)], fontsize=22, fontfamily='serif')
    ax.set_yticklabels([round(i,1) for i in ax.get_yticks()], fontsize=22, fontfamily='serif')
    ax.tick_params(labelsize=22)
    ax.grid(True)
    ax.legend(prop={'size': 18, 'family': 'serif'}, loc='center', bbox_to_anchor=(0.5, -0.15), ncol=2)

In [10]:
results = {}

In [ ]:
errors = predict(semester_data, recommendations.sim_distance)
results['Euclidean Distance'] = errors
plot_errors(errors, 'Euclidean Distance')

In [12]:
import json
with open('2017 paper.json', 'w') as fw:
    json.dump(results, fw)

### subset (no fallback) algorithm 

In [12]:
import json
import numpy as np

# Maskeyi Yükle
with open('subset_fallback_mask.json', 'r') as f:
    valid_mask = json.load(f)

# Bu fonksiyon orijinal 'get_errors' fonksiyonunun Maske eklenmiş halidir.
def get_errors_masked(train_semester, test_semester, sim, current_sem_name, mask):
    y_true = []
    y_pred = []
    gpa = {}
    
    # GPA hesapları (Orijinal koddan)
    for student in train_semester:
        gpa[student] = get_gpa(train_semester, student)
    
    # Her öğrenci için tahmin
    for student in train_semester:
        # Testte yoksa atla
        if student not in test_semester:
            continue
            
        # Orijinal öneri fonksiyonu (recommendations kütüphanesi kullanılıyor)
        # Kodun orijinalinde 'sim' parametresi kullanılıyordu
        recs = recommendations.getRecommendations(train_semester, student, sim)
        
        # Önerileri sözlüğe çevir
        recommended_courses = {rec_course: rec_grade for rec_grade, rec_course in recs}
        
        # Testteki dersleri kontrol et
        for course_code in test_semester[student]:
            
            # --- MASKE KONTROLÜ (BURASI YENİ) ---
            is_valid = False
            student_str = str(student)
            
            if current_sem_name in mask:
                if student_str in mask[current_sem_name]:
                    if course_code in mask[current_sem_name][student_str]:
                        is_valid = True
            
            # Eğer bizim modelimiz bunu tahmin edemediyse, SOTA karşılaştırmasına alma
            if not is_valid:
                continue
            # ------------------------------------

            # Tahmin Mantığı (2017 Paper'ın orijinal mantığı)
            final_pred = None
            
            if course_code in recommended_courses:
                final_pred = recommended_courses[course_code]
            else:
                # Fallback mekanizması (Paper'da varsa GPA kullanılıyordu sanırım)
                final_pred = gpa[student]
            
            # Prediction listesine ekle
            y_pred.append(final_pred)
            y_true.append(test_semester[student][course_code])
            
    return y_true, y_pred

def predict_masked(semester_data, sim, mask):
    errors = {'rmse': [], 'mae': []}
    sorted_semesters = sorted(semester_data.keys()) # Semester isimleri
    
    train_semester = {}
    # 1. dönemden başlayarak time-series split
    for i in range(1, len(sorted_semesters)):
        curr_sem_name = sorted_semesters[i]
        prev_sem_name = sorted_semesters[i-1]
        
        # Train verisini güncelle (Geçmiş dönemleri biriktir)
        new_semester = semester_data[prev_sem_name]
        for student in new_semester:
            if student in train_semester:
                train_semester[student].update(new_semester[student])
            else:
                train_semester[student] = new_semester[student]
        
        test_semester = semester_data[curr_sem_name]

        # MASKELİ Hata Hesabı
        y_true, y_pred = get_errors_masked(train_semester, test_semester, sim, curr_sem_name, mask)

        if len(y_true) > 0:
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            mae = mean_absolute_error(y_true, y_pred)
        else:
            rmse, mae = 0, 0
            
        errors['rmse'].append(rmse)
        errors['mae'].append(mae)
        print(f"Sem {curr_sem_name}: RMSE {rmse}")
    
    return errors

# --- ÇALIŞTIR ---
# 2017 Paper normalde 'recommendations.sim_distance' veya 'sim_pearson' kullanır.
# Dosyada hangisi varsa onu kullan.
results_2017 = predict_masked(semester_data, recommendations.sim_distance, valid_mask)

# Kaydet (Manuel JSON oluşturma çünkü output formatı farklı)
final_rmse = np.mean([r for r in results_2017['rmse'] if r > 0])
final_mae = np.mean([m for m in results_2017['mae'] if m > 0])

results_2017_json = {
    "Model": "2017 Paper (CF) Subset",
    "RMSE": final_rmse,
    "MAE": final_mae
}

with open("results_2017_subset.json", 'w') as f:
    json.dump(results_2017_json, f)

print(f"2017 Paper Subset RMSE: {final_rmse}")

Sem 2011 - Spring: RMSE 0.7709849647620349
Sem 2012 - Fall: RMSE 0.9779763412595672
Sem 2012 - Spring: RMSE 0.9090761466776879
Sem 2013 - Fall: RMSE 0.9564223197641758
Sem 2013 - Spring: RMSE 0.9187280011003733
Sem 2014 - Fall: RMSE 0.9685761333748786
Sem 2014 - Spring: RMSE 1.1099610982728476
2017 Paper Subset RMSE: 0.9445321436016523
